In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib
from pandas.io import sql
from sqlalchemy import create_engine
from matplotlib import font_manager, rc
%matplotlib inline

font_name = matplotlib.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
matplotlib.rc('font', family=font_name)

In [ ]:
#PM2.5 not included
'''
@ 강원도, 경기도, 경상남도, 경상북도, 광주광역시, 대구광역시, 대전광역시, 부산광역시
  서울특별시, 울산광역시, 인천광역시, 전라남도, 전라북도, 제주도, 충청남도, 충청북도

@ 도로이동오염원, 비도로이동오염원, 비산업 연소, 생산공정, 에너지산업 연소, 제조업 연소, 폐기물 처리
'''
def Output2010(df=pd.DataFrame()):
    for filename in os.listdir(""):
        data = pd.read_csv(""%filename, encoding="utf-8")
        data.columns=['Year','Location','Output','CO','NOx','SOx','TSP','PM10','PM2','VOC','NH3']
        # PreProcessing
        data.CO=data.CO.str.replace(',','')
        data.NH3=data.NH3.str.replace(',','')
        data.NOx=data.NOx.str.replace(',','')
        data.SOx=data.SOx.str.replace(',','')
        data.TSP=data.TSP.str.replace(',','')
        data.PM10=data.PM10.str.replace(',','')
        data.VOC=data.VOC.str.replace(',','')
    
        data=data.convert_objects(convert_numeric=True)
        del data['PM2']
        df=df.append(data)
    df.index = range(1, len(df)+1)
    return df

#PM2.5 included
def Output2013(df = pd.DataFrame()):
    for filename in os.listdir(""):
        data = pd.read_csv(""%filename, encoding="utf-8")
        data.columns=['Year','Location','Output','CO','NOx','SOx','TSP','PM10','PM2','VOC','NH3']
        # PreProcessing
        data.CO=data.CO.str.replace(',','')
        data.NH3=data.NH3.str.replace(',','')
        data.NOx=data.NOx.str.replace(',','')
        data.SOx=data.SOx.str.replace(',','')
        data.TSP=data.TSP.str.replace(',','')
        data.PM10=data.PM10.str.replace(',','')
        data.VOC=data.VOC.str.replace(',','')
        data.PM2=data.PM2.str.replace(',','')
        data=data.convert_objects(convert_numeric=True)
        df=df.append(data)
    df.index = range(1, len(df)+1)
    return df

def sqlInclude_2010(data_2010=Output2010()):
    cnData=pd.read_pickle('년도별 미세먼지데이터.pickle')
    
    # Data Mapping
    cnData.columns=['Location','Year','SO2','CO','O3','NO2','PM10_Value']
    cnData['Location']=cnData['Location'].map({'강원':'강원도','경기':'경기도','경남':'경상남도','경북':'경상북도','광주':'광주광역시','대구':'대구광역시', \
                            '대전':'대전광역시','부산':'부산광역시','서울':'서울특별시', '울산':'울산광역시','인천':'인천광역시', \
                            '전남':'전라남도', '전북':'전라북도','제주':'제주도','충남':'충청남도','충북':'충청북도'})
    
    # Mean GroupBy
    cnData=np.round(cnData.groupby(['Location','Year'])['SO2','CO','O3','NO2','PM10_Value'].mean(),4)
    cnData=cnData.reset_index()
    
    # Read OutputData
    dbData=pd.DataFrame(cnData, columns=['Location','Year','PM10_Value'])
    dbData['Year']=dbData['Year'].astype('int64')
    
    # Object to Int64
    dbData_2010=dbData[~dbData['Year'].isin(['2015','2014'])]
    
    # Merge DataFrame
    dbData_2010=data_2010.merge(dbData_2010, how='left', on=['Year','Location'])  
    return dbData_2010

def sqlInclude_2013(data_2013=Output2013()):
    cnData=pd.read_pickle('년도별 미세먼지데이터.pickle')

    cnData.columns=['Location','Year','SO2','CO','O3','NO2','PM10_Value']
    cnData['Location']=cnData['Location'].map({'강원':'강원도','경기':'경기도','경남':'경상남도','경북':'경상북도','광주':'광주광역시','대구':'대구광역시', \
                            '대전':'대전광역시','부산':'부산광역시','서울':'서울특별시', '울산':'울산광역시','인천':'인천광역시', \
                            '전남':'전라남도', '전북':'전라북도','제주':'제주도','충남':'충청남도','충북':'충청북도'})

    # Mean GroupBy
    cnData=np.round(cnData.groupby(['Location','Year'])['SO2','CO','O3','NO2','PM10_Value'].mean(),4)
    cnData=cnData.reset_index()
    
    # Read OutputData
    dbData=pd.DataFrame(cnData, columns=['Location','Year','PM10_Value'])
    dbData_2013=dbData[dbData['Year'].isin(['2011','2012','2013'])]
    
    # Object to Int64
    # Why Error? TT
    dbData_2013['Year']=dbData_2013['Year'].astype('int64')
    
    dbData_2013=data_2013.merge(dbData_2013, how='left', on=['Year','Location'])
    return dbData_2013
    
#correlation
def Output2010_corr(df,Output, Location):
    corr_data=df[df['Output'].isin([Output])]
    corr_data=corr_data[corr_data['Location'].isin([Location])]
    #corr=corr_data.groupby(['Output','Year','Location'])['CO','NOx','SOx','TSP','PM10','VOC','NH3','PM10_Value'].mean()
    return corr_data

def Output2013_corr(df, Output, Location):
    corr_data=df[df['Output'].isin([Output])]
    corr_data=corr_data[corr_data['Location'].isin([Location])]
    #corr=corr_data.groupby(['Output','Year','Location'])['CO','NOx','SOx','TSP','PM10','PM2','VOC','NH3','PM10_Value'].mean()
    return corr_data

In [ ]:
test=Output2010_corr(sqlInclude_2010() , '제조업 연소', '서울특별시')
test.corr()

In [ ]:
'''
@ 강원도, 경기도, 경상남도, 경상북도, 광주광역시, 대구광역시, 대전광역시, 부산광역시
  서울특별시, 울산광역시, 인천광역시, 전라남도, 전라북도, 제주도, 충청남도, 충청북도

@ 도로이동오염원, 비도로이동오염원, 비산업 연소, 생산공정, 에너지산업 연소, 제조업 연소, 폐기물처리
'''
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=1.5)
f, ax = plt.subplots(figsize=(20,10))
ax=sns.heatmap(test,annot=True, fmt=".2f", linewidths=.5, cmap="RdBu_r")